In [2]:
import lzma
import json
import pandas as pd
from collections import Counter

#import gzip
#gzip.GzipFile()

f = lzma.LZMAFile('ecommerce-events.ndjson.xz')

#for l in f:
#    print(json.loads(l.decode('utf-8')))
#    break

events = [json.loads(l.decode('utf-8')) for l in f]

In [3]:
#print(Counter([x['eventType'] for x in events]))

transactions = [x for x in events if x['eventType'] == "transaction"]

print(transactions[0])

{'eventType': 'transaction', 'date': '2017-06-03 06:19:08', 'visitor': '0f0b47a0-f5a3-11e5-b7a4-4ff0e59e724a', 'deviceType': 'desktop', 'osType': 'Windows', 'osVersion': None, 'browserType': 'Chrome', 'browserVersion': '58.0.3029', 'referrerType': None, 'utm_source': None, 'utm_medium': None, 'utm_campaign': None, 'pageType': 'confirmation', 'category': None, 'url': '88ec4ebe923107cfcd385160d1d7d5c4121f8077', 'referrer': '6e03ee62984224d0c0f08d4b68b819297d7f4d14', 'order': 5258, 'orderItems': [{'product': 6461, 'price': 467.91, 'quantity': 2.0}]}


In [4]:
# type(transactions)
df = pd.DataFrame(transactions)
# df.columns = ['visitor','order','event','product','price','quantity']
# columns=['visitor','order','event','product','price','quantity']
df1 = df.drop(['browserType', 'browserVersion', 'category', 'date', 'deviceType', 'eventType', 'osType', 'osVersion', 
        'pageType', 'referrer','referrerType', 'url', 'utm_campaign', 'utm_medium', 'utm_source', 'visitor'], axis=1)
df1.head()

,order,orderItems
0,5258,"[{'product': 6461, 'price': 467.91, 'quantity'..."
1,5385,"[{'product': 19168, 'price': 30.27, 'quantity'..."
2,5547,"[{'product': 18143, 'price': 30.87, 'quantity'..."
3,5355,"[{'product': 19925, 'price': 59.67, 'quantity'..."
4,5717,"[{'product': 38, 'price': 129.9, 'quantity': 1..."


In [5]:
#df2 = pd.DataFrame(df1['orderItems'].values.tolist(), columns=['products','price', 'quantity'])
#,type(df1.orderItems)

In [6]:
transactions2 = []
for t in transactions:
    totalPrice = totalItems = totalQty = 0
    for p in t["orderItems"]:
        totalPrice += p["price"] * p["quantity"]
        totalQty += p["quantity"]
        totalItems += 1
    tnew = {
        #"date": t["date"],
        "order": t["order"],
        "totalItems": totalItems,
        "totalQty": totalQty,
        "price": totalPrice,
    }
    transactions2.append(tnew)
df2 = pd.DataFrame(transactions2)
df2.head()

,order,price,totalItems,totalQty
0,5258,935.82,1,2.0
1,5385,302.70,1,10.0
2,5547,679.14,1,22.0
3,5355,119.34,1,2.0
4,5717,129.90,1,1.0


In [7]:
df3 = df2.groupby(['order'], as_index=False).sum()
df3.price.sum()

212022.27

### Questao 1) resposta: 212022.27

In [14]:
# visitor_device = pd.DataFrame([(e['visitor'], e['deviceType']) for e in events], 
#    columns=['visitor','deviceType'])
# teste = events['deviceType'].nunique()
# teste
mobile_device = [x for x in events if x['deviceType'] == 'mobile']

# print(events[0])

print(mobile_device[0])


{'eventType': 'page', 'date': '2017-06-03 00:07:07', 'visitor': '70b09b50-4808-11e7-b949-31f33cf8565d', 'deviceType': 'mobile', 'osType': 'Android', 'osVersion': '5.1.1', 'browserType': 'Chrome Mobile', 'browserVersion': '58.0.3029', 'referrerType': None, 'utm_source': None, 'utm_medium': None, 'utm_campaign': None, 'pageType': 'subcategory', 'category': None, 'url': 'db2c1fd6668d7432a3e298b40f7b8758cc3e8447', 'referrer': '2282902993188bb2b91ce4e931c6f7a4a946f725'}


In [16]:
df4 = pd.DataFrame(mobile_device)
len(df4)

287358

In [29]:
mobile_device = pd.DataFrame([(e['visitor'], e['deviceType']) for e in events])
mobile_device.drop_duplicates().head()

,0,1
0,89206c00-3f26-11e7-9978-fdec7f5865bc,desktop
1,b96acfe0-4809-11e7-9b7d-e57b4710e964,desktop
2,70b09b50-4808-11e7-b949-31f33cf8565d,mobile
3,ff14b670-4776-11e7-8c72-5f5d3fe5c610,desktop
4,fad427f0-d767-11e6-aea4-61a60e8da6e5,desktop


In [21]:
len(mobile_device.drop_duplicates())

114447

In [24]:
114447/287358

0.3982732340843129

### Questao2) resposta: 0.4

In [26]:
busca = [x for x in events if x['eventType'] == 'search']

print(busca[0])

{'eventType': 'search', 'date': '2017-06-03 00:11:16', 'visitor': '7e7b19a0-b6a3-11e6-8940-2b661c4cb0ac', 'deviceType': 'mobile', 'osType': 'Android', 'osVersion': '4.4.4', 'browserType': 'Chrome', 'browserVersion': '58.0.3029', 'referrerType': 'search', 'utm_source': None, 'utm_medium': None, 'utm_campaign': None, 'pageType': 'search', 'category': None, 'url': 'a4eb2586a7e400d9a76f43ade03453faa5f9c86d', 'referrer': '85ad0be7bba37946a4ef53708c662d4927d04f10', 'query': 'hiper metros standart', 'searchItems': [17470, 8383, 17320, 6117, 6404, 8149, 1162, 88, 409, 13755, 4815, 11465, 1500, 13425, 13872, 13841, 7104, 17990, 11888, 12641, 10815, 7268, 16782, 2991]}


 passo1) filtrar pra pegar apenas as buscas que tiveram pelo menos 15 ocorrências